# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from API_secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 53467702, 'calculationPrice': 'close', 'change': -4.21, 'changePercent': -0.02461, 'close': 166.89, 'closeSource': 'official', 'closeTime': 1698350400061, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 166.83, 'delayedPriceTime': 1698350381995, 'extendedChange': 0.36, 'extendedChangePercent': 0.00216, 'extendedPrice': 167.25, 'extendedPriceTime': 1698364798902, 'high': 171.377, 'highSource': '15 minute delayed price', 'highTime': 1698350399998, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 166.77, 'iexCloseTime': 1698350399930, 'iexLastUpdated': 1698350399930, 'iexMarketPercent': 0.02652170134373173, 'iexOpen': 170.6, 'iexOpenTime': 1698327000643, 'iexRealtimePrice': 166.77, 'iexRealtimeSize': 100, 'iexVolume': 1873102, 'lastTradeTime': 1698350399970, 'latestPrice': 166.89, 'latestSource': 'Close', 'latestTime': 'October 26, 2023', 'latestUpdate': 1698350400061, 'latestVolume': 70625258, 'low': 165.67, 'lowSource': 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
# Create a new DataFrame with the current data
new_row = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns=my_columns)

# Concatenate the new DataFrame to final_dataframe
final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    
    # Get the relevant data from the API
    price = data['latestPrice']
    market_cap = data['marketCap']
    
    # Create a new DataFrame with the current data
    new_row = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns=my_columns)
    
    # Concatenate the new DataFrame to final_dataframe
    final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
final_dataframe

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            # Create a new DataFrame with the current data
            new_row = pd.DataFrame([[symbol,
                                     data[symbol]['quote']['latestPrice'], 
                                     data[symbol]['quote']['marketCap'], 
                                     'N/A']], columns=my_columns)
            
            # Concatenate the new DataFrame to final_dataframe
            final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
            
        except:
            # Tratar a situação em que o símbolo não é encontrado
            print(f"O símbolo {symbol} não foi encontrado ou não retornou dados.")
final_dataframe

O símbolo ABC não foi encontrado ou não retornou dados.
O símbolo ANTM não foi encontrado ou não retornou dados.
O símbolo DISCA não foi encontrado ou não retornou dados.
O símbolo FBHS não foi encontrado ou não retornou dados.
O símbolo HFC não foi encontrado ou não retornou dados.
O símbolo NLOK não foi encontrado ou não retornou dados.
O símbolo RE não foi encontrado ou não retornou dados.
O símbolo VIAC não foi encontrado ou não retornou dados.
O símbolo WLTW não foi encontrado ou não retornou dados.


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,104.31,30519778447,N/A
1,AAL,11.15,7286977133,N/A
2,AAP,51.03,3034767419,N/A
3,AAPL,166.89,2609196978480,N/A
4,ABBV,145.20,256284777936,N/A
...,...,...,...,...
491,YUM,118.75,33275089619,N/A
492,ZBH,103.12,21548381498,N/A
493,ZBRA,204.83,10515637098,N/A
494,ZION,30.45,4511019300,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: